In [1]:
import boto3
import sagemaker
import json

s3_bucket = 'cmg-sagemaker-compliance-cases-data'
s3_prefix = 'inference_data_trigger/'
pipeline_name = 'inference-without-pipinstall-2023-07-25-11-53-15'
region = 'eu-west-2'
role_arn =  sagemaker.get_execution_role()

eventbridge_client = boto3.client('events', region_name=region)
sagemaker_client = boto3.client('sagemaker', region_name=region)

# Get the pipeline ARN
pipeline_arn = sagemaker_client.list_pipelines(PipelineNamePrefix=pipeline_name)['PipelineSummaries'][0]['PipelineArn']

# Create the event pattern
event_pattern = {
    "source": ["aws.s3"],
    "detail-type": ["ObjectCreated"],
    "detail": {
        "eventSource": ["s3.amazonaws.com"],
        "eventName": ["ObjectCreated:Put"],
        "requestParameters": {
            "bucketName": [s3_bucket],
            "key": [{
                "prefix": s3_prefix
            }]
        }
    }
}

# Create the rule
rule_response = eventbridge_client.put_rule(
    Name='cmg_sagemaker_pipeline',
    State='ENABLED',
    Description='Rule to trigger SageMaker pipeline when new file is added to S3 bucket',
    RoleArn=role_arn,
    EventPattern=json.dumps(event_pattern)
)

# Add the target
target_response = eventbridge_client.put_targets(
    Rule='cmg_sagemaker_pipeline',
    Targets=[
        {
            'Id': '1',
            'Arn': pipeline_arn,
            'RoleArn': role_arn,
            'Input': '{"PipelineExecutionDescription": "Started by EventBridge rule"}',
            'InputPath': '$.detail'
        }
    ]
)

print(f'Rule ARN: {rule_response["RuleArn"]}')


ClientError: An error occurred (ValidationException) when calling the PutRule operation: Provided role 'arn:aws:iam::798861612849:role/service-role/AmazonSageMaker-ExecutionRole-20221020T100790' cannot be assumed by principal 'events.amazonaws.com'.

In [9]:
role_arn =  sagemaker.get_execution_role()
print(role_arn)

arn:aws:iam::798861612849:role/service-role/AmazonSageMaker-ExecutionRole-20221020T100790


In [5]:
ROLE_NAME = 'AmazonSageMaker-ExecutionRole-20221020T100790'
!aws iam update-assume-role-policy --role-name ROLE_NAME --policy-document file://trust-policy.json



Error parsing parameter '--policy-document': Unable to load paramfile file://trust-policy.json: [Errno 2] No such file or directory: 'trust-policy.json'


In [7]:
!aws iam get-role --role-name ROLE_NAME --query 'Role.AssumeRolePolicyDocument'



An error occurred (AccessDenied) when calling the GetRole operation: User: arn:aws:sts::798861612849:assumed-role/AmazonSageMaker-ExecutionRole-20221020T100790/SageMaker is not authorized to perform: iam:GetRole on resource: role ROLE_NAME because no identity-based policy allows the iam:GetRole action
